In [33]:
from pathlib import Path

import geopandas as gpd

## Load input layers


In [2]:
sufosat: gpd.GeoDataFrame = gpd.read_file("../data/sufosat/sufosat_clear_cuts_2024.fgb")
sufosat

,date_min,date_max,days_delta,clear_cut_group_size,area_ha,geometry
0,2024-04-21,2024-07-08,78,12,0.500123,"MULTIPOLYGON (((1242170.295 6021778.341, 12421..."
1,2024-03-04,2024-04-21,48,19,1.386440,"MULTIPOLYGON (((1233121.944 6028390.495, 12331..."
2,2024-06-20,2024-07-14,24,7,0.617602,"MULTIPOLYGON (((1198216.849 6067996.198, 11982..."
3,2024-04-08,2024-07-25,108,51,2.102984,"MULTIPOLYGON (((1163543.691 6133084.278, 11635..."
4,2024-05-26,2024-07-14,49,6,0.509373,"MULTIPOLYGON (((1188128.497 6151090.369, 11881..."
...,...,...,...,...,...,...
11661,2024-03-11,2024-05-22,72,15,0.571641,"MULTIPOLYGON (((322563.449 6225453.651, 322563..."
11662,2024-04-10,2024-06-03,54,21,1.025675,"MULTIPOLYGON (((320038.024 6221675.317, 320038..."
11663,2024-05-10,2024-05-22,12,4,0.532672,"MULTIPOLYGON (((313513.415 6220771.654, 313514..."
11664,2024-02-28,2024-06-27,120,80,4.974293,"MULTIPOLYGON (((325514.216 6204393.711, 325514..."


In [3]:
natura2000: gpd.GeoDataFrame = gpd.read_file("../data/natura2000").rename(
    columns={"Couches": "type", "SITECODE": "code", "SITENAME": "name"}
)[["type", "name", "geometry"]]
natura2000

,type,name,geometry
0,ZPS,Etang de Saint Quentin,"POLYGON ((627623.168 6854888.067, 627620.379 6..."
1,ZPS,Massif de Fontainebleau,"MULTIPOLYGON (((686304.526 6806792.984, 686243..."
2,ZPS,Massif de Villefermoy,"MULTIPOLYGON (((699069.343 6818948.132, 698740..."
3,ZPS,Bassée et plaines adjacentes,"MULTIPOLYGON (((729943.243 6825377.977, 730620..."
4,ZPS,Boucles de la Marne,"MULTIPOLYGON (((714834.051 6873595.418, 714786..."
...,...,...,...
1757,ZSC,Montagnes du Barétous,"MULTIPOLYGON (((400290.475 6227215.38, 400583...."
1758,ZSC,Massif des Arbailles,"MULTIPOLYGON (((370428.867 6238736.854, 370575..."
1759,ZSC,Montagnes de la Haute Soule,"MULTIPOLYGON (((383161.014 6223904.758, 383354..."
1760,ZSC,Posidonies de la côte palavasienne,"POLYGON ((764008.157 6255745.243, 762998.071 6..."


In [4]:
slope30: gpd.GeoDataFrame = gpd.read_file("../data/ign/bdalti25/slope_gte_30.fgb")
slope30

,geometry
0,"POLYGON ((1225687.5 6062937.5, 1225687.5 60629..."
1,"POLYGON ((1225587.5 6062937.5, 1225587.5 60629..."
2,"POLYGON ((1225612.5 6063462.5, 1225662.5 60634..."
3,"POLYGON ((1225512.5 6062912.5, 1225512.5 60628..."
4,"POLYGON ((1225637.5 6062712.5, 1225637.5 60626..."
...,...
764107,"POLYGON ((313637.5 6263537.5, 313637.5 6263487..."
764108,"POLYGON ((313612.5 6264112.5, 313637.5 6264112..."
764109,"POLYGON ((313462.5 6263987.5, 313512.5 6263987..."
764110,"POLYGON ((313812.5 6264437.5, 313812.5 6264412..."


## Stich the input layers together


### Natura 2000


In [ ]:
# ~1 minute
# TODO: This should have be done in a prior data preparation step

# For now, we just union_all all the areas for simplicity
natura2000 = gpd.GeoDataFrame(geometry=[natura2000.union_all()], crs=natura2000.crs)
# Explode the multipolygon to get the individual polygons (one for each site)
natura2000 = natura2000.explode()
natura2000

,geometry
0,"POLYGON ((317766.398 6260743.551, 317798.001 6..."
0,"POLYGON ((321087.848 6259077.852, 321050.71 62..."
0,"POLYGON ((321049.916 6259506.28, 321061.857 62..."
0,"POLYGON ((222235.151 6425916.243, 220496.621 6..."
0,"POLYGON ((350925.078 6599683.636, 350930.265 6..."
...,...
0,"POLYGON ((1226164.86 6195358.743, 1226164.928 ..."
0,"POLYGON ((1219473.509 6196094.104, 1219450.895..."
0,"POLYGON ((1226717.332 6211898.684, 1226684.19 ..."
0,"POLYGON ((1217249.6 6229586.4, 1217250 6229586..."


In [6]:
# ~2 minutes

# Join Sufosat and Natura2000
natura2000["natura2000_geometry"] = natura2000["geometry"]
sufosat: gpd.GeoDataFrame = gpd.sjoin(sufosat, natura2000, how="left", predicate="intersects")

# One Sufosat clear cut can join with several Natura2000 polygons
# We group the gdf by unique index values (unique clear cuts), take the "first" value for all columns (as they're duplicates),
# except for the natura2000 geometry that we union
sufosat = (
    sufosat.set_geometry("natura2000_geometry")
    .dissolve(by=sufosat.index, aggfunc="first", method="unary")
    .set_geometry("geometry")
    .set_crs(epsg=2154)
)

# Get the area of the intersection
sufosat["natura2000_area_ha"] = (
    sufosat["geometry"].intersection(sufosat["natura2000_geometry"]).area / 10000
)

# Get rid of useless columns that come from the spatial join
sufosat = sufosat.drop(columns=["index_right", "natura2000_geometry"])

sufosat

,date_min,date_max,days_delta,clear_cut_group_size,area_ha,geometry,natura2000_area_ha
0,2024-04-21,2024-07-08,78,12,0.500123,"MULTIPOLYGON (((1242170.295 6021778.341, 12421...",0.0
1,2024-03-04,2024-04-21,48,19,1.386440,"MULTIPOLYGON (((1233121.944 6028390.495, 12331...",0.0
2,2024-06-20,2024-07-14,24,7,0.617602,"MULTIPOLYGON (((1198216.849 6067996.198, 11982...",0.0
3,2024-04-08,2024-07-25,108,51,2.102984,"MULTIPOLYGON (((1163543.691 6133084.278, 11635...",0.0
4,2024-05-26,2024-07-14,49,6,0.509373,"MULTIPOLYGON (((1188128.497 6151090.369, 11881...",0.0
...,...,...,...,...,...,...,...
11661,2024-03-11,2024-05-22,72,15,0.571641,"MULTIPOLYGON (((322563.449 6225453.651, 322563...",0.0
11662,2024-04-10,2024-06-03,54,21,1.025675,"MULTIPOLYGON (((320038.024 6221675.317, 320038...",0.0
11663,2024-05-10,2024-05-22,12,4,0.532672,"MULTIPOLYGON (((313513.415 6220771.654, 313514...",0.0
11664,2024-02-28,2024-06-27,120,80,4.974293,"MULTIPOLYGON (((325514.216 6204393.711, 325514...",0.0


### Slope >= 30%


In [7]:
# ~4-5 minutes

# TODO: use multiprocessing to speed this up?

# Join Sufosat and IGN's slope
slope30["slope30_geometry"] = slope30["geometry"]
sufosat: gpd.GeoDataFrame = gpd.sjoin(sufosat, slope30, how="left", predicate="intersects")

# Get the area of the intersection
sufosat["slope30_area_ha"] = (
    sufosat["geometry"].intersection(sufosat["slope30_geometry"]).area / 10000
)

# A clear cut can join several slope polygons, we want to only keep the largest one
sufosat = sufosat.sort_values("slope30_area_ha", ascending=False)
sufosat = sufosat[~sufosat.index.duplicated(keep="first")]

# Fill NaN values (if there is no intersection with a slope30 area)
sufosat["slope30_area_ha"] = sufosat["slope30_area_ha"].fillna(0)

# Get rid of useless columns that come from the spatial join
sufosat = sufosat.drop(columns=["index_right", "slope30_geometry"])

sufosat

,date_min,date_max,days_delta,clear_cut_group_size,area_ha,geometry,natura2000_area_ha,slope30_area_ha
11440,2024-01-19,2024-08-04,198,232,18.992690,"MULTIPOLYGON (((621971.434 6205475.203, 621971...",0.0,18.193203
1624,2024-03-03,2024-04-08,36,23,14.641623,"MULTIPOLYGON (((1058091.351 6352671.595, 10580...",0.0,13.426039
9491,2024-05-30,2024-08-04,66,137,15.143128,"MULTIPOLYGON (((652839.11 6323247.987, 652839....",0.0,10.348887
1755,2024-05-02,2024-05-14,12,16,11.277782,"MULTIPOLYGON (((1004010.877 6679036.791, 10040...",0.0,9.266934
1800,2024-03-27,2024-04-08,12,9,7.309445,"MULTIPOLYGON (((1007184.323 6701820.652, 10071...",0.0,6.386112
...,...,...,...,...,...,...,...,...
11661,2024-03-11,2024-05-22,72,15,0.571641,"MULTIPOLYGON (((322563.449 6225453.651, 322563...",0.0,0.000000
11662,2024-04-10,2024-06-03,54,21,1.025675,"MULTIPOLYGON (((320038.024 6221675.317, 320038...",0.0,0.000000
11663,2024-05-10,2024-05-22,12,4,0.532672,"MULTIPOLYGON (((313513.415 6220771.654, 313514...",0.0,0.000000
11664,2024-02-28,2024-06-27,120,80,4.974293,"MULTIPOLYGON (((325514.216 6204393.711, 325514...",0.0,0.000000


### Add centroïd & representation point information

The centroid is the geometric center of a shape, but it may lie outside the geometry, while the representative point is always inside the geometry and is used when you need a point guaranteed to be within the shape.


In [24]:
# TODO: This should have be done in a prior data preparation step

# I don't simply do `sufosat["centroid"] = sufosat.geometry` because `gdf.to_file()`
# doesn't support multiple geometries
sufosat["centroid_x"] = sufosat.geometry.centroid.x
sufosat["centroid_y"] = sufosat.geometry.centroid.y
sufosat["representative_point_x"] = sufosat.geometry.representative_point().x
sufosat["representative_point_y"] = sufosat.geometry.representative_point().y

## Abusive clear cuts


In [26]:
# Reorder columns
sufosat = sufosat[
    [
        "date_min",
        "date_max",
        "days_delta",
        "clear_cut_group_size",
        "area_ha",
        "natura2000_area_ha",
        "slope30_area_ha",
        "geometry",
        "centroid_x",
        "centroid_y",
        "representative_point_x",
        "representative_point_y",
    ]
]

# Sort by date
sufosat = sufosat.sort_values("date_min").reset_index(drop=True)

sufosat

,date_min,date_max,days_delta,clear_cut_group_size,area_ha,natura2000_area_ha,slope30_area_ha,geometry,centroid_x,centroid_y,representative_point_x,representative_point_y
0,2024-01-01,2024-03-13,72,14,0.783329,0.000000,0.000000,"MULTIPOLYGON (((908017.689 6977592.177, 908009...",908047.813183,6.977602e+06,908021.624030,6.977603e+06
1,2024-01-01,2024-02-18,48,23,0.952374,0.000000,0.000000,"MULTIPOLYGON (((673518.255 6174334.842, 673518...",673570.659058,6.174315e+06,673591.979077,6.174315e+06
2,2024-01-01,2024-03-25,84,14,0.565434,0.565434,0.000000,"MULTIPOLYGON (((830280.84 6591727.152, 830271....",830319.547934,6.591749e+06,830335.322640,6.591764e+06
3,2024-01-01,2024-04-23,113,57,3.051765,0.000000,0.000000,"MULTIPOLYGON (((736865.871 6548920.436, 736865...",736979.821843,6.548855e+06,736949.836775,6.548835e+06
4,2024-01-01,2024-03-25,84,13,0.584156,0.584156,0.000000,"MULTIPOLYGON (((817539.826 6589473.5, 817539.2...",817581.534500,6.589529e+06,817589.405527,6.589530e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
11661,2024-08-04,2024-08-04,0,1,1.269001,0.000000,0.003268,"MULTIPOLYGON (((808950.141 6550898.506, 808950...",809020.382833,6.550836e+06,809020.747758,6.550839e+06
11662,2024-08-04,2024-08-04,0,1,0.679622,0.000000,0.000000,"MULTIPOLYGON (((816583.742 6564810.989, 816583...",816582.513639,6.564779e+06,816570.558232,6.564775e+06
11663,2024-08-04,2024-08-04,0,1,0.764027,0.000000,0.000000,"MULTIPOLYGON (((813595.996 6633936.694, 813595...",813551.105118,6.633883e+06,813574.009341,6.633890e+06
11664,2024-08-04,2024-08-04,0,1,0.640599,0.000000,0.000000,"MULTIPOLYGON (((806803.791 6669634.929, 806803...",806833.547542,6.669525e+06,806828.217835,6.669540e+06


In [27]:
# Abusive clear cuts with an area >= 10 hectares
sufosat[sufosat["area_ha"] >= 10]

,date_min,date_max,days_delta,clear_cut_group_size,area_ha,natura2000_area_ha,slope30_area_ha,geometry,centroid_x,centroid_y,representative_point_x,representative_point_y
43,2024-01-01,2024-07-29,210,226,15.066794,0.000000,3.011971,"MULTIPOLYGON (((654182.025 6263780.205, 654182...",654175.605604,6.263682e+06,654299.669253,6.263945e+06
110,2024-01-02,2024-04-01,90,299,21.656049,21.656049,0.000000,"MULTIPOLYGON (((821753.681 6289661.137, 821753...",821862.391841,6.289735e+06,821873.524812,6.289462e+06
181,2024-01-04,2024-08-03,212,191,13.479794,0.000000,0.000000,"MULTIPOLYGON (((400515.95 6390681.534, 400516....",400729.300316,6.390963e+06,400655.845825,6.390916e+06
208,2024-01-05,2024-07-27,204,218,19.181661,0.000000,0.000000,"MULTIPOLYGON (((356552.784 6304484.515, 356553...",356878.378809,6.304382e+06,356820.980691,6.304436e+06
298,2024-01-07,2024-06-11,156,141,11.767085,0.000382,0.000000,"MULTIPOLYGON (((468417.083 6386141.93, 468417....",468157.612027,6.386333e+06,468194.373785,6.386306e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
9738,2024-06-15,2024-08-02,48,80,12.267579,0.000000,0.000000,"MULTIPOLYGON (((348349.487 6303934.768, 348349...",348594.255033,6.303937e+06,348610.201767,6.303926e+06
9760,2024-06-15,2024-08-03,49,119,11.643299,0.390185,0.000000,"MULTIPOLYGON (((462466.889 6599030.176, 462466...",462670.945617,6.599309e+06,462686.085063,6.599336e+06
9974,2024-06-21,2024-08-03,43,87,10.500394,0.000000,0.000000,"MULTIPOLYGON (((367544.441 6351902.694, 367525...",367654.343340,6.351940e+06,367589.167423,6.351932e+06
10259,2024-06-27,2024-08-03,37,78,12.596832,12.596832,0.000000,"MULTIPOLYGON (((432921.421 6431708.372, 432920...",433173.477041,6.431813e+06,433177.243662,6.431824e+06


In [28]:
# Abusive clear cuts within a Natura2000 area
(
    len(sufosat[sufosat["natura2000_area_ha"] >= 0.5]),
    len(sufosat[sufosat["natura2000_area_ha"] >= 1]),
    len(sufosat[sufosat["natura2000_area_ha"] >= 2]),
)

(1144, 525, 193)

In [29]:
# Abusive clear cuts with a slope >= 30%
(
    len(sufosat[sufosat["slope30_area_ha"] >= 0.5]),
    len(sufosat[sufosat["slope30_area_ha"] >= 1]),
    len(sufosat[sufosat["slope30_area_ha"] >= 2]),
)

(473, 198, 67)

In [30]:
# Abusive clear cuts with all criterias
abusive_clear_cuts: gpd.GeoDataFrame = sufosat[
    (sufosat["area_ha"] >= 10)
    | (sufosat["natura2000_area_ha"] >= 2)
    | (sufosat["slope30_area_ha"] >= 2)
]
abusive_clear_cuts

,date_min,date_max,days_delta,clear_cut_group_size,area_ha,natura2000_area_ha,slope30_area_ha,geometry,centroid_x,centroid_y,representative_point_x,representative_point_y
43,2024-01-01,2024-07-29,210,226,15.066794,0.000000,3.011971,"MULTIPOLYGON (((654182.025 6263780.205, 654182...",654175.605604,6.263682e+06,654299.669253,6.263945e+06
96,2024-01-02,2024-03-02,60,43,2.973299,2.973299,0.000000,"MULTIPOLYGON (((854310.569 6283384.677, 854310...",854413.715365,6.283468e+06,854434.527618,6.283463e+06
99,2024-01-02,2024-08-05,216,71,6.054324,6.054324,0.000000,"MULTIPOLYGON (((644046.22 6427631.19, 644036.7...",644166.058184,6.427657e+06,644164.693320,6.427660e+06
101,2024-01-02,2024-03-02,60,79,4.479064,4.479064,0.000000,"MULTIPOLYGON (((847784.044 6277194.569, 847784...",847836.043068,6.277178e+06,847776.718655,6.277093e+06
103,2024-01-02,2024-02-19,48,23,2.388063,2.388063,1.587221,"MULTIPOLYGON (((787376.363 6357724.191, 787357...",787413.311190,6.357691e+06,787265.500367,6.357787e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
11182,2024-07-16,2024-07-29,13,13,2.194105,2.194105,0.000000,"MULTIPOLYGON (((579683.157 6624865.08, 579683....",579813.798935,6.624858e+06,579838.520180,6.624847e+06
11196,2024-07-16,2024-07-29,13,12,2.173247,2.173247,0.000000,"MULTIPOLYGON (((598790.754 6682717.737, 598772...",598872.066902,6.682856e+06,598854.178291,6.682847e+06
11314,2024-07-21,2024-08-02,12,10,2.079933,2.079933,0.000000,"MULTIPOLYGON (((507069.934 6849252.297, 507052...",506993.296131,6.849330e+06,507041.425650,6.849324e+06
11405,2024-07-22,2024-08-03,12,5,3.056744,3.056744,0.000000,"MULTIPOLYGON (((430367.687 6354103.764, 430369...",430398.157982,6.354270e+06,430406.862394,6.354235e+06


## Save the result to S3

We leave the criterias filtering up to the consumers of the file


In [36]:
# Save the result to disk
# index=False because we don't need to keep the group id
Path("../data/abusive_clear_cuts/").mkdir(exist_ok=True, parents=True)
sufosat.to_file("../data/abusive_clear_cuts/abusive_clear_cuts_2024.fgb", index=False)

In [35]:
# Upload to s3
!aws s3 cp ../data/abusive_clear_cuts/abusive_clear_cuts_2024.fgb s3://brigade-coupe-rase-s3/analytics/data/abusive_clear_cuts/abusive_clear_cuts_2024.fgb --profile d4g-s13-brigade-coupes-rases

upload: ../data/abusive_clear_cuts/abusive_clear_cuts_2024.fgb to s3://brigade-coupe-rase-s3/analytics/data/abusive_clear_cuts/abusive_clear_cuts_2024.fgb


In [38]:
!aws s3 ls s3://brigade-coupe-rase-s3/analytics/ --recursive --profile d4g-s13-brigade-coupes-rases

2025-02-25 09:31:33   96946219 analytics/data
2025-02-28 13:19:58  121412416 analytics/data/abusive_clear_cuts/abusive_clear_cuts_2024.fgb
2025-02-21 18:43:11  558882864 analytics/data/ign/bdalti25/slope_gte_30.fgb
2025-02-21 18:42:55   32993275 analytics/data/ign/bdalti25/slope_gte_30.tif
2025-02-25 09:58:17   96946219 analytics/data/sufosat/mosaics_tropisco_warnings_france_date.tif
2025-02-25 10:08:57   76601154 analytics/data/sufosat/mosaics_tropisco_warnings_france_prob.tif
2025-02-21 13:26:51  120737768 analytics/data/sufosat/sufosat_clear_cuts_2024.fgb
